In [1]:
from tensorflow.keras import models, layers, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_base = '../../data/res_data/base/'
path_novel2 = '../../data/res_data/novel2/'
prob = []

In [5]:
import tensorflow as tf
gpu_id = 0
print(tf.__version__)
if tf.__version__ >= "2.1.0":
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.list_physical_devices('GPU')
    tf.config.set_visible_devices(physical_devices[gpu_id], 'GPU')
    tf.config.experimental.set_memory_growth(physical_devices[gpu_id], True)

2.3.1


In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="./train/",
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="./train/",
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./test/",
x_col="id",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

In [ ]:
num_0_train = len(os.listdir(path_train+'0'))
num_1_train = len(os.listdir(path_train+'1'))

num_0_val = len(os.listdir(path_val+'0'))
num_1_val = len(os.listdir(path_val+'1'))

total_train = num_0_train + num_1_train
total_val = num_0_val + num_1_val

print(total_train, total_val)

batch_size = 32
epochs = 10
IMG_HEIGHT = 32
IMG_WIDTH = 32

train_image_generator = ImageDataGenerator(rescale=1./255) # 学習データのジェネレータ
val_image_generator = ImageDataGenerator(rescale=1./255) # 検証データのジェネレータ
# predict_image_generator = ImageDataGenerator(rescale=1./255) # 予測データのジェネレータ

train_data_gen = train_image_generator.flow_from_directory(directory=path_train,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = val_image_generator.flow_from_directory(directory=path_val,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

input_tensor = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=input_tensor, pooling='avg'
)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(1, activation='sigmoid'))


model = models.Model(inputs=base_model.input, outputs=top_model(base_model.output))

from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])

# model.summary()




history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# with open('../data/acc/natural2(synthetic).txt', mode='a') as f:
#     f.write(str(val_acc[-1]) + '\n')

for l in model.layers[:-1]:
    l.trainable = False
# model.summary()

path_train = path_novel2+'tr/'
path_val = path_novel2+'val/'

num_0_train = len(os.listdir(path_train+'0'))
num_1_train = len(os.listdir(path_train+'1'))

num_0_val = len(os.listdir(path_val+'0'))
num_1_val = len(os.listdir(path_val+'1'))

total_train = num_0_train + num_1_train
total_val = num_0_val + num_1_val

train_image_generator = ImageDataGenerator(rescale=1./255) # 学習データのジェネレータ
val_image_generator = ImageDataGenerator(rescale=1./255) # 検証データのジェネレータ
# predict_image_generator = ImageDataGenerator(rescale=1./255) # 予測データのジェネレータ

train_data_gen = train_image_generator.flow_from_directory(directory=path_train,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = val_image_generator.flow_from_directory(directory=path_val,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')


history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# with open('../data/acc/natural2(natural).txt', mode='a') as f:
#     f.write(str(val_acc[-1]) + '\n')
prob.append(val_acc[-1])